# 基于MNIST训练集生成Universal adversarial perturbations


In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.python.framework import graph_util
import os

pb_file_path="tensorflow_mnist_graph.pb"

g = tf.Graph()
with tf.Session(graph=g) as sess:

    mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)
    in_units = 784 #输入节点数
    h1_units = 300 #隐含层节点数

    #初始化隐含层权重W1，服从默认均值为0，标准差为0.1的截断正态分布
    W1 = tf.Variable(tf.truncated_normal([in_units, h1_units], stddev=0.1)) 
    b1 = tf.Variable(tf.zeros([h1_units])) #隐含层偏置b1全部初始化为0
    W2 = tf.Variable(tf.zeros([h1_units, 10])) 
    b2 = tf.Variable(tf.zeros([10]))
    x = tf.placeholder(tf.float32, [None, in_units],name="input")
    keep_prob = tf.placeholder(tf.float32,name="keep_prob") 

    #定义模型结构
    hidden1 = tf.nn.relu(tf.matmul(x, W1) + b1)
    hidden1_drop = tf.nn.dropout(hidden1, keep_prob)
    y = tf.nn.softmax(tf.matmul(hidden1_drop, W2) + b2,name="output")

    #训练部分
    y_ = tf.placeholder(tf.float32, [None, 10])
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
    train_step = tf.train.AdagradOptimizer(0.3).minimize(cross_entropy)


    sess.run(tf.global_variables_initializer())
    correct_prediction = tf.equal(tf.arg_max(y, 1), tf.arg_max(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    writer = tf.summary.FileWriter('/tmp/logs/', sess.graph)
    
    
    for i in range(5000):
        batch_xs, batch_ys = mnist.train.next_batch(100) 
        _,loss=sess.run([train_step,cross_entropy],{x: batch_xs, y_: batch_ys, keep_prob: 0.75})
        
        if i % 200 == 0:
            acc=accuracy.eval(feed_dict={x:mnist.test.images,y_:mnist.test.labels,keep_prob:1})
            print("loss={},acc={}".format(loss,acc))


    constant_graph = graph_util.convert_variables_to_constants(sess, sess.graph_def, output_node_names=["output"])

    with tf.gfile.FastGFile(pb_file_path, mode='wb') as f:
        f.write(constant_graph.SerializeToString())

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Use `argmax` instead
loss=2.3025851249694824,acc=0.25519999861717224
loss=0.21335826814174652,acc=0.9430999755859375
loss=0.27882134914398193,acc=0.9559999704360962
loss=0.14894792437553406,acc=0.9674000144004822
loss=0.08424727618694305,acc=0.9729999899864197
loss=0.036558